In [11]:
#Importing all the neccessary libraries
import pandas as pd
import numpy as np #for linear algebra

#Libraries for visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

#Libraries for formattting and handling text 
import string
import re

#Library for nltk
import nltk 
import nltk.corpus
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud

from collections import defaultdict
from collections import  Counter
stop=set(stopwords.words('english'))

#Library for NN
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from keras.initializers import Constant


import sklearn
#Library for Splitting Dataset
from sklearn.model_selection import train_test_split
#Library for TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection, naive_bayes
#Library for evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report

import warnings
warnings.filterwarnings("ignore")

import plotly
plotly.offline.init_notebook_mode (connected = True)

import neattext as nt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sridevi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sridevi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sridevi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
# oly considering train.csv

df = pd.read_csv("train.csv")
df.head(5)
x = df["text"]
y = df["target"]
print(type(x))

<class 'pandas.core.series.Series'>


In [13]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)
print(x_train.shape, x_val.shape)

(6090,) (1523,)


In [14]:
def clean_data(text):
    clean_pipeline = nt.TextPipeline(steps=[nt.normalize, nt.remove_numbers, nt.fix_contractions, nt.remove_stopwords, nt.remove_emails, nt.remove_emojis, nt.remove_html_tags, nt.remove_urls, nt.remove_special_characters])
    text = clean_pipeline.transform(text)
 
    lemmatizer = WordNetLemmatizer()
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    text = " ".join([t for t in text.split() if len(t) > 3 ])
    return text

x_train_clean= x_train.apply(clean_data)
print(x_train_clean.head(2))

x_val_clean = x_val.apply(clean_data)
print(x_val_clean.head(2))

4996    courageous honest analysis need atomic bomb ja...
3263    zachzaidman shame golf cart engulfed flame boy...
Name: text, dtype: object
2644                weapon cause unimaginable destruction
2227    famping thing gishwhes soaked deluge going tam...
Name: text, dtype: object


In [15]:
tfidf_vect = TfidfVectorizer(max_features=5000)
tfidf_vect.fit(x_train)

x_train_tfidf = tfidf_vect.transform(x_train)
x_val_tfidf = tfidf_vect.transform(x_val)

In [16]:
count_vect = CountVectorizer()
count_vect.fit(x_train)

x_train_count_vect = count_vect.transform(x_train)
x_val_count_vect = count_vect.transform(x_val)

In [17]:
def model_exec(x_train, y_train, x_val, y_val, models=None):
    if models is None:
        models = [
              DecisionTreeClassifier(),
              GradientBoostingClassifier(),
              sklearn.naive_bayes.MultinomialNB(), 
              sklearn.linear_model.LogisticRegression(),
              sklearn.svm.SVC(),
              sklearn.svm.SVC(C=10),
              sklearn.svm.SVC(C=0.5),
            ]
    else:
        if not isinstance(models, list):
            models = list(models)
            
    accuracy_models = []
    for model in models:
        model.fit(x_train, y_train)
        predictions = model.predict(x_val)
        print("Accuracy Score -> ", model , accuracy_score(predictions, y_val)*100)
        accuracy_models.append((accuracy_score(predictions, y_val)*100, model))
        
    print(accuracy_models)
    return max(accuracy_models)

In [18]:
# using tfidf
max_accuracy, best_model = model_exec(x_train_tfidf, y_train, 
                                      x_val_tfidf, y_val)
print(best_model, max_accuracy)

Accuracy Score ->  DecisionTreeClassifier() 71.24097176625082
Accuracy Score ->  GradientBoostingClassifier() 76.23112278397899
Accuracy Score ->  MultinomialNB() 80.76165462902168
Accuracy Score ->  LogisticRegression() 79.97373604727511
Accuracy Score ->  SVC() 81.22127380170716
Accuracy Score ->  SVC(C=10) 79.97373604727511
Accuracy Score ->  SVC(C=0.5) 80.17071569271175
[(71.24097176625082, DecisionTreeClassifier()), (76.23112278397899, GradientBoostingClassifier()), (80.76165462902168, MultinomialNB()), (79.97373604727511, LogisticRegression()), (81.22127380170716, SVC()), (79.97373604727511, SVC(C=10)), (80.17071569271175, SVC(C=0.5))]
SVC() 81.22127380170716


In [9]:
# using count_vectorizer
max_accuracy, best_model = model_exec(x_train_count_vect, y_train, 
                                      x_val_count_vect, y_val)
print(best_model, max_accuracy)

Accuracy Score ->  DecisionTreeClassifier() 72.94812869336835
Accuracy Score ->  GradientBoostingClassifier() 75.90282337491793
Accuracy Score ->  MultinomialNB() 80.10505581089954
Accuracy Score ->  LogisticRegression() 80.9586342744583
Accuracy Score ->  SVC() 81.08995403808274
Accuracy Score ->  SVC(C=10) 80.23637557452396
Accuracy Score ->  SVC(C=0.5) 78.52921864740644
[(72.94812869336835, DecisionTreeClassifier()), (75.90282337491793, GradientBoostingClassifier()), (80.10505581089954, MultinomialNB()), (80.9586342744583, LogisticRegression()), (81.08995403808274, SVC()), (80.23637557452396, SVC(C=10)), (78.52921864740644, SVC(C=0.5))]
SVC() 81.08995403808274
